In [1]:
# desc
!pip install uuid requests pandas rdflib requests SPARQLWrapper pydantic

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 8.6 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 11.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 9.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=a5ed5af544b67eee43f572d692bdf06f9751becdf5e36aade24e5505572a2356
  Stored in directory: /home/jovyan/.cache/pip/wheels/cb/ba/bc/7ae4822f65dea25d41ebe0cf219cd7ddfd0987634750ee41b3
Successfully built uuid


In [1]:
#test

In [2]:
import json
import uuid

import requests

# import the smartpynector module
import smartpynector as sp


In [3]:
# Defining constants
# Knowledge engine REST API URL
# KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
KNOWLEDGE_ENGINE_URL = "http://knowledge_engine:8280/rest"
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"


In [4]:
# Defining the Smart Connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=THERMOSTAT_API_URL,
    knowledgeBaseName="thermostat 1",
    knowledgeBaseDescription="thermostat 1",
    reasonerEnabled=False,
)

In [5]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-13 21:07:48 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:07:50 DEBUG http://knowledge_engine:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-13 21:07:50 INFO Request successful
2023-05-13 21:07:50 DEBUG 


<Response [200]>

In [6]:

GRAPH_PATTERN = '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
}
HEADERS = {
    "Content-Type": "application/json",
    "knowledge-Base-Id": THERMOSTAT_API_URL,
}


In [7]:
answer_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AnswerKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers=HEADERS,
).json()["knowledgeInteractionId"]

answer_ki_id

2023-05-13 21:07:50 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:07:50 DEBUG http://knowledge_engine:8280 "POST /rest/sc/ki HTTP/1.1" 200 108
2023-05-13 21:07:50 INFO Request successful
2023-05-13 21:07:50 DEBUG {"knowledgeInteractionId":"http://0.0.0.0:8001/thermostat/interaction/c946e535-502d-4ccd-978f-c10008ee2fb1"}


'http://0.0.0.0:8001/thermostat/interaction/c946e535-502d-4ccd-978f-c10008ee2fb1'

In [8]:
thermostat_actual_temperature = requests.get(THERMOSTAT_API_URL + '/actual_temperature').json()
thermostat_actual_temperature

2023-05-13 21:07:50 DEBUG Starting new HTTP connection (1): 0.0.0.0:8001
2023-05-13 21:07:50 DEBUG http://0.0.0.0:8001 "GET /thermostat/actual_temperature HTTP/1.1" 200 2


20

In [9]:
# %%timeit
sp.perform_answer_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=THERMOSTAT_API_URL,
    knowledge_interaction_id=answer_ki_id,
    answer_binding_set=[{
        "meas": f'<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>',
        "temp": f"{thermostat_actual_temperature}",
        "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
        "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
        "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
    }]
)

2023-05-13 21:07:50 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:01 DEBUG http://knowledge_engine:8280 "GET /rest/sc/handle HTTP/1.1" 200 208
2023-05-13 21:08:01 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:01 DEBUG http://knowledge_engine:8280 "POST /rest/sc/handle HTTP/1.1" 200 0


<Response [200]>

In [10]:
# Lets try implementing a POST / REACT knowledge interaction

In [11]:
react_ki = sp.register_knowledge_interaction(
    'ReactKnowledgeInteraction', KNOWLEDGE_ENGINE_URL, PREFIXES, headers= HEADERS,
    argument_graph_pattern="?a rdf:type saref:Measurement . ?a saref:hasValueInMeters ?b .",
    result_graph_pattern="?a rdf:type saref:Measurement . ?a saref:hasValueInYards ?b .",
)
react_ki.json()['knowledgeInteractionId']

2023-05-13 21:08:01 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:01 DEBUG http://knowledge_engine:8280 "POST /rest/sc/ki HTTP/1.1" 200 108
2023-05-13 21:08:01 INFO Request successful
2023-05-13 21:08:01 DEBUG {"knowledgeInteractionId":"http://0.0.0.0:8001/thermostat/interaction/61006776-400c-4eba-8da3-ee2a54ced3e2"}


'http://0.0.0.0:8001/thermostat/interaction/61006776-400c-4eba-8da3-ee2a54ced3e2'